<a href="https://colab.research.google.com/github/annabellesauve/Empathy-Detection/blob/main/CISC867_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
! pip install transformers
! pip install tensorflow_addons

In [121]:
import tensorflow as tf
import tensorflow_addons as tfa
from transformers import RobertaTokenizer, TFRobertaModel, TFRobertaForMaskedLM
import pandas as pd
import numpy as np
import math

Roberta documentation: https://huggingface.co/transformers/model_doc/roberta.html?highlight=roberta#tfrobertamodel

# Encoders

In [122]:
# robertabase model initialization, from huggingface
robertabase_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
robertabase_model = TFRobertaModel.from_pretrained('roberta-base')
robertabase_maskedLM_model = TFRobertaForMaskedLM.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at roberta-b

**S Encoder**

In [138]:
# def S_Encoder(seeker_post, id_SP, attention_SP):
class S_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  # def _init_weights(self, module):
  #   # don't know what weights to use to initialize.. not much info 
  #   module.bias.data.zero_()
  #   module.weight.data.fill_(1.0)
  
  def encode(self, seeker_post):
    encoded_input = robertabase_tokenizer(seeker_post, return_tensors='tf')
    return robertabase_model(encoded_input)
    # input_ids = np.array(robertabase_tokenizer.encode(seeker_post, add_special_tokens=True))
    # return robertabase_maskedLM_model(input_ids, labels=input_ids)
    # inputs = robertabase_tokenizer(seeker_post_mask, return_tensors="tf")
    # inputs["labels"] = robertabase_tokenizer(seeker_post, return_tensors="tf")["input_ids"]
    # return robertabase_maskedLM_model(inputs)

**R Encoder**

In [124]:
class R_Encoder(tf.Module):
  def __init__(self):
    super().__init__()

  # def _init_weights(self, module):
  #   # don't know what weights to use to initialize.. not much info 
  #   module.bias.data.zero_()
  #   module.weight.data.fill_(1.0)

  def encode(self, response_post):
    encoded_input = robertabase_tokenizer(response_post, return_tensors='tf')
    return robertabase_model(encoded_input)

  # def encode_mask(response_post, response_post_mask):

  #   inputs = robertabase_tokenizer(response_post_mask, return_tensors="tf")["input_ids"]
  #   inputs["labels"] = robertabase_tokenizer(response_post, return_tensors="tf")["input_ids"]
  #   return robertabase_maskedLM_model(inputs)
    

# Attention Layer

In [127]:
# query = ei(R)
# keys + values = ei(S)

# attention(i)(ei(R), ei(S)) = softmax((ei(R)*ei(S))/sqrt(d))*ei(S)
# d = 768

# residual mapping hi(R) = ei(R) + a(i)(ei(R), ei(S))

In [177]:
class AttentionLayer(tf.Module):
  
  def __init__(self):
    super().__init__()

  def attention(self, e_R, e_S, d):
    a = tf.math.multiply(tf.nn.softmax(tf.math.divide(tf.math.multiply(e_R, e_S), math.sqrt(d))), e_R)
    return a



## dont know where this is used   
def residual(e_R, e_S, d):
  h = e_R + attention(e_R, e_S, d)
  return h

# Models

In [178]:
# three models for three communication mechanisms in EPITOME

# emotional reaction
# interpretation
# exploration

# BiEncoderAttentionWithRationaleClassification() in models.py
class Model(tf.Module):

  def __init__(self, hidden_dropout_prob=0.2, rationale_num_labels=2, empathy_num_labels= 3, hidden_size=768):
    super().__init__()

    self.hidden_dropout_prob = hidden_dropout_prob
    self.rationale_classifier = tf.keras.layers.Dense(rationale_num_labels, activation=None)
    self.attention = AttentionLayer()
    self.rationale_num_labels = rationale_num_labels
    self.empathy_num_labels = empathy_num_labels
    self.empathy_classifier = () # need to figure what this is 
    self.r_encoder = R_Encoder()
    self.s_encoder = S_Encoder()
    self.hidden_size = hidden_size

    # self.apply(self._init_weights)

  def _init_weights(self, module):
    # don't know what weights to use to initialize.. not much info 
    module.bias.data.zero_()
    module.weight.data.fill_(1.0)
  
  def forward(self,
		input_ids_SP=None,
		input_ids_RP=None,
		attention_mask_SP=None,
		attention_mask_RP=None,
		token_type_ids_SP=None,
		token_type_ids_RP=None,
		position_ids_SP=None,
		position_ids_RP=None,
		head_mask_SP=None,
		head_mask_RP=None,
		inputs_embeds_SP=None,
		inputs_embeds_RP=None,
		empathy_labels=None,
		rationale_labels=None,
		lambda_EI=1,
		lambda_RE=0.1,
    SP=None,
    RP=None
    ):

    output_SP = []
    output_RP = []

    for post in SP:
      output_SP.append(self.s_encoder.encode(post))
    
    for post in RP:
      output_RP.append(self.r_encoder.encode(post))

    output = []

    # print(output_RP[0])
    for i in range(len(output_RP)):
      attn = self.attention.attention(output_RP[i].pooler_output, output_SP[i].pooler_output, self.hidden_size)
      output.append(tf.nn.dropout(attn, self.hidden_dropout_prob))

    logits_empathy = self.empathy_classifier(output[:, 0, :])

    output = dropout(output)
    logits_rationales = self.rationale_classifier(output)

    loss_function = tf.keras.losses.CategoricalCrossentropy()

    loss_empathy = loss_function(logits_empathy.reshape(-1, empathy_num_labels), empathy_labels.reshape(-1))
    loss_rationales = loss_function(logits_rationales.reshape(-1, rationale_num_labels), rationale_labels.reshape(-1))

    overall_loss = lambda_E1 * loss_empathy + lambda_RE + loss_rationales

    return (overall_loss, loss_empathy, loss_rationales, logits_empathy, logits_rationale)

# Empathy Identification

In [169]:
# pass representation of CLS token in residual response post h through linear layer to get empathy level (0, 1, 2)
# do this for three models for three communication mechanisms in EPITOME

# figure out how they get this in the actual model (not used right now)

# EmpathyClassifier() in empathy_classifier.py
class EmpathyClassifier():

  def __init__(self):
    self.tokenizer = robertabase_tokenizer
    self.batch_size = 1

    self.model_ER = Model()
    self.model_IP = Model()
    self.model_EX = Model() 

  def predict_empathy(self, seeker_posts, response_posts): 

    id_SP = [] # ids 
    id_RP = [] 
    attention_SP = [] # attention masks
    attention_RP = []

    # code provided on paper's github, adapted for tensorflow
    for sent in seeker_posts: # tokenize the seeker posts
      encoded_dict = self.tokenizer.encode_plus(
          sent, 
          add_special_tokens = True, # add CLS and SEP tokens
          max_length = 64,
          pad_to_max_length = True,
          return_attention_mask = True,
          return_tensors = 'tf' # tensorflow tensors
      )

      id_SP.append(encoded_dict['input_ids'])
      attention_SP.append(encoded_dict['attention_mask'])
    
    for sent in response_posts: # tokenize response posts
      encoded_dict = self.tokenizer.encode_plus(
          sent, 
          add_special_tokens = True, # add CLS and SEP tokens
          max_length = 64,
          pad_to_max_length = True,
          return_attention_mask = True,
          return_tensors = 'tf' # tensorflow tensore
      )

      id_RP.append(encoded_dict['input_ids'])
      attention_RP.append(encoded_dict['attention_mask'])

    id_SP = tf.concat(id_SP, 0)
    id_RP = tf.concat(id_RP, 0)
    attention_SP = tf.concat(attention_SP, 0)
    attention_RP = tf.concat(attention_RP, 0)

    # create dataset
    dataset = tf.data.Dataset.from_tensors(input_SP, attention_SP, input_RP, attention_RP)

    # evaluate
    for element in dataset:
      e_id_SP = element[0] # get seeker id 
      e_attention_SP = element[1] # get seeker attention 
      e_id_RP = element[2] # get response id 
      e_attention_RP = element[3] # get response attention

      # _loss, _loss_e, _loss_r, logits_empathy_ER, logits_rationale_ER = self.model_ER(e_id_SP, e_attention_SP, e_id_RP, e_attention_RP, seeker_posts, response_posts)
      # _loss, _loss_e, _loss_r, logits_empathy_IP, logits_rationale_IP = self.model_IP(e_id_SP, e_attention_SP, e_id_RP, e_attention_RP, seeker_posts, response_posts)
      # _loss, _loss_e, _loss_r, logits_empathy_EX, logits_rationale_EX = self.model_EX(e_id_SP, e_attention_SP, e_id_RP, e_attention_RP, seeker_posts, response_posts)

      prediction_ER = np.argmax(logits_empathy_ER, axis=1).flatten()
      prediction_IP = np.argmax(logits_empathy_IP, axis=1).flatten()
      prediction_EX = np.argmax(logits_empathy_EX, axis=1).flatten()
    
    return (logits_empathy_ER, predictions_ER, logits_empathy_IP, predictions_IP, logits_empathy_EX, predictions_EX)

# Rationale Extraction

In [131]:
# pass representation of individual tokens in residual response post h through linear layer to make boolean prediction 
# also do this for three models for three communication mechanisms in EPITOME

# i think this happens under model()

# Initializing model + training

In [132]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [150]:
# load dataset
  # assuming for now seeker posts is "seeker_posts" and reponse posts is "response_posts"
df_ER = pd.read_csv('/content/drive/My Drive/emotionalreactions.csv')
df_ER = df_ER.head(50) # temporary

df_ER['response_post_masked'] = df_ER['response_post_masked'].replace(np.nan, 0)

In [134]:
# roberta domain adaptive pre training 3 epoch 8 batch

# can't get this to work. the tfrobertaformaskedlm example doesn't even work.

# optimizer_roberta = tfa.optimizers.AdamW(
#     weight_decay=1e-2, # not given but default in py torch so probably used
#     learning_rate=2e-5, # given
#     epsilon=1e-8 # given
# )

# sp = df_ER.seeker_post 
# rp = df_ER.response_post
# rp_masked = df_ER.response_post_masked

# encoder_train_data = [sp, rp, rp_masked]

# S_encoder_model = S_Encoder()
# R_encoder_model = R_Encoder()

# total_encoder_train_loss = 0
# total_encoder_sp_loss = 0
# total_encoder_rp_loss = 0

# for epoch in range(0, 3):
#   print("Training epoch #", epoch, "...")

#   for i in range(len(encoder_train_data[0])):
#       e_sp = encoder_train_data[0][i] # seeker post 
#       e_rp = encoder_train_data[1][i] # response post 
#       e_rp_masked = encoder_train_data[2][i] # masked response post

#       # sp_output = S_Encoder.encode(e_sp)
#       if not isinstance(e_rp_masked, str):
#         continue 
#       else:
#         rp_output = R_Encoder.encode_mask(e_rp, e_rp_masked)

#         print(rp_output)
#         total_encoder_train_loss += rp_output.loss # what is total loss? no info
#         # total_encoder_sp_loss += sp_output.loss
#         total_encoder_rp_loss += rp_output.loss

#         optimizer_roberta.minimize(rp_output.loss) # update optimizer

#         print(rp_output.loss)
    
#   print("Epoch #", epoch)
#   print("Total loss:", total_train_loss)
#   print("Total empathy loss:", total_train_empathy_loss)
#   print("Total rationale loss:", total_train_rationale_loss)

In [179]:
# tokenize input 
tokenizer_RP = robertabase_tokenizer.batch_encode_plus(df_ER.response_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
input_ids_RP = np.array(tokenizer_RP['input_ids'])
attention_masks_RP = np.array(tokenizer_RP['attention_mask'])

tokenizer_SP = robertabase_tokenizer.batch_encode_plus(df_ER.seeker_post, add_special_tokens=True, max_length=64, padding='max_length', truncation=True)
input_ids_SP = np.array(tokenizer_SP['input_ids'])
attention_masks_SP = np.array(tokenizer_RP['attention_mask'])

labels = np.array(df_ER.level.values.astype(int))
rationales = np.array(df_ER.rationale_labels.values.tolist())

ER_model = Model()
# IN_model = Model()
# EX_model = Model()

optimizer_model = tfa.optimizers.AdamW(
    weight_decay=1e-2, # not given but default in py torch so probably used
    learning_rate=2e-5, # given
    epsilon=1e-8 # given
)

# ER_model.compile(optimizer=optimizer_model)

In [152]:
# train_dataset = tf.data.Dataset(input_ids_SP, attention_masks_SP, input_ids_RP, attention_masks_RP, labels, rationales)
train_dataset = [input_ids_SP, attention_masks_SP, input_ids_RP, attention_masks_RP, labels, rationales]
train_size = int(len(train_dataset))

total_steps = train_size * 4 # 4 is epoch
# num_batch = int(train_size / 32) # 32 is batch size

tf.random.set_seed(12) # given seed

In [176]:
for epoch in range(0, 4):
  print("Training epoch #", epoch, "...")
  total_train_loss = 0
  total_train_empathy_loss = 0
  total_train_rationale_loss = 0

  # ER_model.train()

  for i in range(len(train_dataset[0])):
  # for element in train_dataset:
      e_id_SP = train_dataset[0][i] # get seeker id 
      e_attention_SP = train_dataset[1][i] # get seeker attention 
      e_id_RP = train_dataset[2][i] # get response id 
      e_attention_RP = train_dataset[3][i] # get response attention
      e_labels = train_dataset[4][i] # get labels
      e_rationales = train_dataset[5][i] # get rationales

      loss, loss_empathy, loss_rationale, logits_empathy, logits_rationale = ER_model.forward(
          input_ids_SP=e_id_SP,
          input_ids_RP=e_id_RP,
          attention_mask_SP=e_attention_SP,
          attention_mask_RP=e_attention_RP,
          empathy_labels=e_labels,
          rationale_labels=e_rationales,
          SP=df_ER.seeker_post,
          RP=df_ER.response_post
          )
      
      total_train_loss += loss
      total_train_empathy_loss += loss_empathy
      total_train_rationale_loss += loss_rationale

      optimizer_model.minimize(loss) # update optimizer

      print(loss)
    
  print("Epoch #", epoch)
  print("Total loss:", total_train_loss)
  print("Total empathy loss:", total_train_empathy_loss)
  print("Total rationale loss:", total_train_rationale_loss)

Training epoch # 0 ...
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divide
softmax
other mul
attention
here
mul
divid

TypeError: ignored